In [ ]:
#Load the required packages
import numpy as np
import pandas as pd

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

#Read the csv file
url = 'https://raw.githubusercontent.com/HOGENT-Databases/BI-BigData/master/data/customers.csv'
dataset=pd.read_csv(url,sep=',')

#Explore the dataset
print(dataset.head()) #top 5 columns
print(len(dataset))  # of rows

#descriptive statistics of the dataset
dataset.describe().transpose()

Since INCOME and SPENT have a different order of magnitude, which might confuse the algoritms, we first apply standard scaling to the input. The Python StandardScaler standardizes a feature by subtracting the mean and then scaling to unit variance. Unit variance means dividing all the values by the standard deviation.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaled_dataset = dataset
scaler = StandardScaler()
income = dataset.drop('SPENT',axis=1)
print(income.head())
spent = dataset.drop('INCOME',axis=1)

print(spent.head())

dataset['INCOME_SC'] = scaler.fit_transform(income)
dataset['SPENT_SC'] = scaler.fit_transform(spent)




In [ ]:
#Using the elbow method to find the optimum number of clusters
from sklearn.cluster import KMeans
wcss = []
for i in range(1,11):
    # k-means++ initializes the centroids to be (generally) distant from each other, 
    # leading to probably better results than random initialization
    km=KMeans(n_clusters=i,init='k-means++')
    km.fit(dataset[['INCOME_SC','SPENT_SC']])
    # inertia = within cluster some of squares (wcss)
    wcss.append(np.round(km.inertia_).astype('int64'))

wcss = pd.Series(wcss)

# Plot the dataset
import matplotlib.pyplot as plt
%matplotlib inline

plt.scatter(range(1,11), wcss)
plt.xlabel('clusters')
plt.ylabel('wcss')

plt.show()

In [ ]:
# Based on the elbow plot, we could choose 4,5 or 6 clusters. 
# Let us try 4, 5 and 6 clusters and visualize the clusters to decide on the final number of clusters.
# Fitting kmeans to the dataset with k=4
km4=KMeans(n_clusters=4,init='k-means++')
y_means = km4.fit_predict(dataset[['INCOME_SC','SPENT_SC']])
dataset['Cluster'] = y_means+1
print(dataset.head())

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15,5
groups = dataset.groupby('Cluster')
print(groups)
fig,ax = plt.subplots()

for name, group in groups:
    ax.plot(group.INCOME, group.SPENT, marker='o', linestyle='', ms=12, label=name)
    
plt.xlabel('INCOME')
plt.ylabel('SPENT')
    
ax.legend()

In [ ]:
km5=KMeans(n_clusters=5,init='k-means++')
y_means = km5.fit_predict(dataset[['INCOME_SC','SPENT_SC']])
dataset['Cluster'] = y_means + 1

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15,5
groups = dataset.groupby('Cluster')
print(groups)
fig,ax = plt.subplots()

for name, group in groups:
    ax.plot(group.INCOME, group.SPENT, marker='o', linestyle='', ms=12, label=name)
    
plt.xlabel('INCOME')
plt.ylabel('SPENT')

ax.legend()

In [ ]:
km6=KMeans(n_clusters=6,init='k-means++')
y_means = km6.fit_predict(dataset[['INCOME_SC','SPENT_SC']])
dataset['Cluster'] = y_means + 1

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15,5
groups = scaled_dataset.groupby('Cluster')
print(groups)
fig,ax = plt.subplots()

for name, group in groups:
    ax.plot(group.INCOME, group.SPENT, marker='o', linestyle='', ms=12, label=name)
    
plt.xlabel('INCOME')
plt.ylabel('SPENT')

ax.legend()

4 clusters seems to be a good starting point: it makes alle combinatios of high spent, low spent, high income and low income. 

#### The importance of scaling
To check the importance of scaling let's create 6 clusters using the original values for income and spent.

In [ ]:
km6=KMeans(n_clusters=6,init='k-means++')
y_means = km6.fit_predict(dataset[['INCOME','SPENT']])
dataset['Cluster'] = y_means + 1

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15,5
groups = scaled_dataset.groupby('Cluster')
print(groups)
fig,ax = plt.subplots()

for name, group in groups:
    ax.plot(group.INCOME, group.SPENT, marker='o', linestyle='', ms=12, label=name)
    
plt.xlabel('INCOME')
plt.ylabel('SPENT')

ax.legend()

We see the algorithm continue splitting along the INCOME axis because these are the larger numbers. Of course this does not make sense.